In [ ]:
import pandas as pd
import numpy as np
import os
print('number of training image')
print(len([name for name in os.listdir('/kaggle/input/plant-pathology-2021-fgvc8/train_images/')]))
print(len([name for name in os.listdir('/kaggle/input/plant-pathology-2021-fgvc8/test_images/')]))
training_csv = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
training_class = np.array([])
for labels in pd.unique(training_csv['labels']):
    training_class = np.append(training_class,labels.split())
training_class = np.unique(training_class)
print('\nnumber of class')
print(training_class)
print(len(training_class))

In [ ]:
def predict2strings(pred, threshold):
    t = np.array(threshold)
    return ' '.join(training_class[np.where(pred >= t)])

def predict2n_hot(pred, threshold):
    t = np.array(threshold)
    return np.where(pred >= t,np.ones(pred.shape),np.zeros(pred.shape))

def n_hot2string(n_hot):
    return ' '.join(training_class[np.where(n_hot >= 1)])

In [ ]:
import tensorflow as tf
model_input = tf.keras.layers.Input(shape=(299, 299, 3))
xception_layer = tf.keras.applications.Xception(
    include_top=False,
    weights=None,
    input_shape= (299, 299, 3),
    input_tensor= model_input
).output
at = tf.keras.layers.Conv2D(2048,3,padding='same',activation = 'relu', )(xception_layer)
at = tf.keras.layers.BatchNormalization()(at)
at = tf.keras.layers.Conv2D(2048,1,padding='same',activation = 'relu')(at)
at = tf.keras.layers.BatchNormalization()(at)
at = tf.keras.layers.Conv2D(1,1,padding='same',activation = 'relu')(at)
attention_heatmap = tf.keras.layers.Softmax(name = 'attention_layer')(at)
attention = tf.keras.layers.Multiply()([attention_heatmap,xception_layer])
fusion_attention_spectrum = tf.keras.layers.Add()([attention,xception_layer])
fusion_attention_spectrum = tf.keras.layers.BatchNormalization()(fusion_attention_spectrum)
fc_1 = tf.keras.layers.Flatten()(fusion_attention_spectrum)
fc_1 = tf.keras.layers.Dense(2048, activation = 'relu')(fc_1)
fc_1 = tf.keras.layers.BatchNormalization()(fc_1)
fc_2 = tf.keras.layers.Dense(4096 + 2048, activation = 'relu')(fc_1)
fc_2 = tf.keras.layers.BatchNormalization()(fc_2)
average_pooling = tf.keras.layers.GlobalAveragePooling2D()(fusion_attention_spectrum)
fc_2 = tf.keras.layers.concatenate([fc_2,average_pooling])
model_output = tf.keras.layers.Dense(len(training_class),activation = 'sigmoid')(fc_2)
model = tf.keras.Model(inputs = model_input,outputs = model_output)

In [ ]:
model.load_weights('../input/fork-of-cs5489-project-train-cont-12/my_model/attention_1')

In [ ]:
threshold =[0.9151549935340881, 0.028693564236164093, 0.22229868173599243, 0.987423300743103, 0.18797093629837036, 0.6855853796005249]
test_img_path = '../input/plant-pathology-2021-fgvc8/test_images/'
def write_csv_kaggle_tags():
    import csv
    tmp = [['image', 'labels']]
    
    for path in os.listdir(test_img_path):
        image = tf.keras.preprocessing.image.load_img(test_img_path + path, target_size=(299,299))
        image = tf.keras.preprocessing.image.img_to_array(image)
        image = tf.image.per_image_standardization(image)
        image = tf.expand_dims(image, axis=0,)
        test_predscore = model(image)[0]
        row = [path, predict2strings(test_predscore, threshold = threshold)]
        tmp.append(row)
    f = open("submission.csv", 'w')
    tmp
    writer = csv.writer(f)
    writer.writerows(tmp)
    f.close()

In [ ]:
write_csv_kaggle_tags()